# <font color='red'>Classification using extracted features</font>
This notebook shows a classification using extracted features. With this it is possible to use typical supervised learning methods such as KNN or Logistic Regression for the classification problem.

## <font color='blue'>Note</font>: This Notebook contains the "all-in-one" classification! For a better and cleaner solution/overview use the modularized Notebooks instead.

<hr>
<center> <h1><font color="blue">Import modules</font></h1> </center>
<hr>

## Import general modules

In [1]:
#import neccessary modules
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import statistics
import matplotlib.cm as cm
%matplotlib inline

## Import modules for feature extraction

In [2]:
# tsfresh modules (for feature extraction)
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters

import logging
# Set logger-level to "error". Not recommed: Important warnings can be overseen
logging.basicConfig(level=logging.ERROR)

## Import modules for the standard supervised learning algorithms

In [3]:
# import machine learning algorithms
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

<hr>
<center> <h1><font color="blue">Import and split the dataset</font></h1> </center>
<hr>
The current dataset can be load as a csv-file. The csv-file is presented as an dataframe object were each time series (punch) is identified with a unique punch_id (continual)

In [4]:
ds = pd.read_csv('../../Datasets/complete_smartpunch_dataset_AsOneDataFrame_periodMS2000_sampleUS1000.csv')
del ds['Unnamed: 0'] # remove the unnecessary column ... maybe this can be fixed in a later version of the dataset-exporter?! 

In [6]:
ds.head(2005)

In [5]:
ds.head()

In [6]:
test_ds= [
    {
        "_id": "5c8faaaae37533453c77fb20",
        "label": "Training",
        "hand": "L+R",
        "annotator": "Trainingsmode",
        "count": 81,
        "periodNS": 767000000,
        "raws": [
            {
                "_id": "5c8faaaae37533453c77facf",
                "timestamp": "0",
                "x": 9.68375301361084,
                "y": 0.9762716293334961,
                "z": 0.9834501147270203
            },
            {
                "_id": "5c8faaaae37533453c77fad0",
                "timestamp": "3000000",
                "x": 9.686145782470703,
                "y": 1.0456634759902954,
                "z": 1.0289137363433838
            },
            {
                "_id": "5c8faaaae37533453c77fad1",
                "timestamp": "1000000",
                "x": 9.645467758178711,
                "y": 1.1102696657180786,
                "z": 1.1341979503631592
            },
            {
                "_id": "5c8faaaae37533453c77fad2",
                "timestamp": "13000000",
                "x": 9.616754531860352,
                "y": 1.184447169303894,
                "z": 1.1676974296569824
            },
            {
                "_id": "5c8faaaae37533453c77fad3",
                "timestamp": "6000000",
                "x": 9.631111145019531,
                "y": 1.2394821643829346,
                "z": 1.1892328262329102
            },
            {
                "_id": "5c8faaaae37533453c77fad4",
                "timestamp": "9000000",
                "x": 9.698110580444336,
                "y": 1.4021940231323242,
                "z": 1.3399807214736938
            },
            {
                "_id": "5c8faaaae37533453c77fad5",
                "timestamp": "11000000",
                "x": 9.67657470703125,
                "y": 1.3854442834854126,
                "z": 1.2299108505249023
            },
            {
                "_id": "5c8faaaae37533453c77fad6",
                "timestamp": "13000000",
                "x": 9.69093132019043,
                "y": 1.4524433612823486,
                "z": 1.1102696657180786
            },
            {
                "_id": "5c8faaaae37533453c77fad7",
                "timestamp": "6000000",
                "x": 9.681360244750977,
                "y": 1.3591232299804688,
                "z": 0.8997012972831726
            },
            {
                "_id": "5c8faaaae37533453c77fad8",
                "timestamp": "13000000",
                "x": 9.69093132019043,
                "y": 1.5409778356552124,
                "z": 0.8542376756668091
            },
            {
                "_id": "5c8faaaae37533453c77fad9",
                "timestamp": "6000000",
                "x": 9.678967475891113,
                "y": 1.6390835046768188,
                "z": 0.8039883971214294
            },
            {
                "_id": "5c8faaaae37533453c77fada",
                "timestamp": "13000000",
                "x": 9.650253295898438,
                "y": 1.5864413976669312,
                "z": 0.763310432434082
            },
            {
                "_id": "5c8faaaae37533453c77fadb",
                "timestamp": "13000000",
                "x": 9.640682220458984,
                "y": 1.648654818534851,
                "z": 0.8111668825149536
            },
            {
                "_id": "5c8faaaae37533453c77fadc",
                "timestamp": "5000000",
                "x": 9.616754531860352,
                "y": 1.674975872039795,
                "z": 0.8327022790908813
            },
            {
                "_id": "5c8faaaae37533453c77fadd",
                "timestamp": "12000000",
                "x": 9.609575271606445,
                "y": 1.7228323221206665,
                "z": 0.8590233325958252
            },
            {
                "_id": "5c8faaaae37533453c77fade",
                "timestamp": "7000000",
                "x": 9.542576789855957,
                "y": 1.7276179790496826,
                "z": 0.8398807644844055
            },
            {
                "_id": "5c8faaaae37533453c77fadf",
                "timestamp": "12000000",
                "x": 9.54496955871582,
                "y": 1.7850457429885864,
                "z": 0.8374879360198975
            },
            {
                "_id": "5c8faaaae37533453c77fae0",
                "timestamp": "7000000",
                "x": 9.516255378723145,
                "y": 1.8376877307891846,
                "z": 0.8422735929489136
            },
            {
                "_id": "5c8faaaae37533453c77fae1",
                "timestamp": "12000000",
                "x": 9.53061294555664,
                "y": 1.8616160154342651,
                "z": 0.8398807644844055
            },
            {
                "_id": "5c8faaaae37533453c77fae2",
                "timestamp": "12000000",
                "x": 9.559326171875,
                "y": 1.9142581224441528,
                "z": 0.7824530005455017
            },
            {
                "_id": "5c8faaaae37533453c77fae3",
                "timestamp": "6000000",
                "x": 9.509077072143555,
                "y": 1.9357935190200806,
                "z": 0.7537391185760498
            },
            {
                "_id": "5c8faaaae37533453c77fae4",
                "timestamp": "11000000",
                "x": 9.528220176696777,
                "y": 1.947757601737976,
                "z": 0.7728816866874695
            },
            {
                "_id": "5c8faaaae37533453c77fae5",
                "timestamp": "8000000",
                "x": 9.54496955871582,
                "y": 1.9453648328781128,
                "z": 0.8039883971214294
            },
            {
                "_id": "5c8faaaae37533453c77fae6",
                "timestamp": "14000000",
                "x": 9.547362327575684,
                "y": 1.9310078620910645,
                "z": 0.8135597109794617
            },
            {
                "_id": "5c8faaaae37533453c77fae7",
                "timestamp": "10000000",
                "x": 9.528220176696777,
                "y": 1.9525432586669922,
                "z": 0.8327022790908813
            },
            {
                "_id": "5c8faaaae37533453c77fae8",
                "timestamp": "7000000",
                "x": 9.521040916442871,
                "y": 1.9381862878799438,
                "z": 0.8327022790908813
            },
            {
                "_id": "5c8faaaae37533453c77fae9",
                "timestamp": "12000000",
                "x": 9.504291534423828,
                "y": 1.8903298377990723,
                "z": 0.849452018737793
            },
            {
                "_id": "5c8faaaae37533453c77faea",
                "timestamp": "5000000",
                "x": 9.523433685302734,
                "y": 1.852044701576233,
                "z": 0.8781659007072449
            },
            {
                "_id": "5c8faaaae37533453c77faeb",
                "timestamp": "11000000",
                "x": 9.521040916442871,
                "y": 1.8233308792114258,
                "z": 0.9260223507881165
            },
            {
                "_id": "5c8faaaae37533453c77faec",
                "timestamp": "9000000",
                "x": 9.549755096435547,
                "y": 1.794616937637329,
                "z": 0.9930213689804077
            },
            {
                "_id": "5c8faaaae37533453c77faed",
                "timestamp": "12000000",
                "x": 9.53779125213623,
                "y": 1.8113667964935303,
                "z": 1.0456634759902954
            },
            {
                "_id": "5c8faaaae37533453c77faee",
                "timestamp": "16000000",
                "x": 9.513862609863281,
                "y": 1.8352949619293213,
                "z": 1.0480563640594482
            },
            {
                "_id": "5c8faaaae37533453c77faef",
                "timestamp": "14000000",
                "x": 9.475577354431152,
                "y": 1.8711873292922974,
                "z": 1.0121639966964722
            },
            {
                "_id": "5c8faaaae37533453c77faf0",
                "timestamp": "12000000",
                "x": 9.456435203552246,
                "y": 1.8855443000793457,
                "z": 0.9044869542121887
            },
            {
                "_id": "5c8faaaae37533453c77faf1",
                "timestamp": "5000000",
                "x": 9.475577354431152,
                "y": 1.94297194480896,
                "z": 0.8350951075553894
            },
            {
                "_id": "5c8faaaae37533453c77faf2",
                "timestamp": "16000000",
                "x": 9.53061294555664,
                "y": 1.9860427379608154,
                "z": 0.8685946464538574
            },
            {
                "_id": "5c8faaaae37533453c77faf3",
                "timestamp": "4000000",
                "x": 9.477970123291016,
                "y": 2.0554347038269043,
                "z": 0.9834501147270203
            },
            {
                "_id": "5c8faaaae37533453c77faf4",
                "timestamp": "1000000",
                "x": 9.485149383544922,
                "y": 2.012363910675049,
                "z": 0.9834501147270203
            },
            {
                "_id": "5c8faaaae37533453c77faf5",
                "timestamp": "12000000",
                "x": 9.542576789855957,
                "y": 2.117648124694824,
                "z": 0.9810572862625122
            },
            {
                "_id": "5c8faaaae37533453c77faf6",
                "timestamp": "6000000",
                "x": 9.516255378723145,
                "y": 2.091326951980591,
                "z": 0.914058268070221
            },
            {
                "_id": "5c8faaaae37533453c77faf7",
                "timestamp": "10000000",
                "x": 9.57607650756836,
                "y": 2.053041934967041,
                "z": 0.8207381963729858
            },
            {
                "_id": "5c8faaaae37533453c77faf8",
                "timestamp": "7000000",
                "x": 9.533005714416504,
                "y": 1.9932212829589844,
                "z": 0.7465606927871704
            },
            {
                "_id": "5c8faaaae37533453c77faf9",
                "timestamp": "11000000",
                "x": 9.564111709594727,
                "y": 2.0482561588287354,
                "z": 0.7561319470405579
            },
            {
                "_id": "5c8faaaae37533453c77fafa",
                "timestamp": "11000000",
                "x": 9.511469841003418,
                "y": 2.069791555404663,
                "z": 0.7441678643226624
            },
            {
                "_id": "5c8faaaae37533453c77fafb",
                "timestamp": "9000000",
                "x": 9.501898765563965,
                "y": 2.100898265838623,
                "z": 0.8685946464538574
            },
            {
                "_id": "5c8faaaae37533453c77fafc",
                "timestamp": "10000000",
                "x": 9.492327690124512,
                "y": 2.0673987865448,
                "z": 0.8422735929489136
            },
            {
                "_id": "5c8faaaae37533453c77fafd",
                "timestamp": "8000000",
                "x": 9.449256896972656,
                "y": 2.069791555404663,
                "z": 0.8015955686569214
            },
            {
                "_id": "5c8faaaae37533453c77fafe",
                "timestamp": "11000000",
                "x": 9.485149383544922,
                "y": 2.122433662414551,
                "z": 0.8805587291717529
            },
            {
                "_id": "5c8faaaae37533453c77faff",
                "timestamp": "13000000",
                "x": 9.45882797241211,
                "y": 2.0961127281188965,
                "z": 0.8231309652328491
            },
            {
                "_id": "5c8faaaae37533453c77fb00",
                "timestamp": "5000000",
                "x": 9.55214786529541,
                "y": 2.1846470832824707,
                "z": 0.9547362327575684
            },
            {
                "_id": "5c8faaaae37533453c77fb01",
                "timestamp": "16000000",
                "x": 9.53779125213623,
                "y": 2.069791555404663,
                "z": 0.849452018737793
            },
            {
                "_id": "5c8faaaae37533453c77fb02",
                "timestamp": "11000000",
                "x": 9.487542152404785,
                "y": 2.0099709033966064,
                "z": 0.6771687865257263
            },
            {
                "_id": "5c8faaaae37533453c77fb03",
                "timestamp": "4000000",
                "x": 9.55214786529541,
                "y": 2.1200408935546875,
                "z": 0.727418065071106
            },
            {
                "_id": "5c8faaaae37533453c77fb04",
                "timestamp": "12000000",
                "x": 9.564111709594727,
                "y": 2.201396942138672,
                "z": 0.7776673436164856
            },
            {
                "_id": "5c8faaaae37533453c77fb05",
                "timestamp": "6000000",
                "x": 9.499505996704102,
                "y": 2.2301106452941895,
                "z": 0.7537391185760498
            },
            {
                "_id": "5c8faaaae37533453c77fb06",
                "timestamp": "15000000",
                "x": 9.55214786529541,
                "y": 2.1750757694244385,
                "z": 0.727418065071106
            },
            {
                "_id": "5c8faaaae37533453c77fb07",
                "timestamp": "6000000",
                "x": 9.504291534423828,
                "y": 2.1822543144226074,
                "z": 0.7250252366065979
            },
            {
                "_id": "5c8faaaae37533453c77fb08",
                "timestamp": "12000000",
                "x": 9.492327690124512,
                "y": 2.2372891902923584,
                "z": 0.7250252366065979
            },
            {
                "_id": "5c8faaaae37533453c77fb09",
                "timestamp": "6000000",
                "x": 9.4660062789917,
                "y": 2.256431818008423,
                "z": 0.6963114142417908
            },
            {
                "_id": "5c8faaaae37533453c77fb0a",
                "timestamp": "12000000",
                "x": 9.494720458984375,
                "y": 2.2396819591522217,
                "z": 0.727418065071106
            },
            {
                "_id": "5c8faaaae37533453c77fb0b",
                "timestamp": "13000000",
                "x": 9.518648147583008,
                "y": 2.2205393314361572,
                "z": 0.7393822073936462
            },
            {
                "_id": "5c8faaaae37533453c77fb0c",
                "timestamp": "6000000",
                "x": 9.506684303283691,
                "y": 2.2205393314361572,
                "z": 0.7417750358581543
            },
            {
                "_id": "5c8faaaae37533453c77fb0d",
                "timestamp": "8000000",
                "x": 9.461220741271973,
                "y": 2.242074728012085,
                "z": 0.7680960893630981
            },
            {
                "_id": "5c8faaaae37533453c77fb0e",
                "timestamp": "10000000",
                "x": 9.45882797241211,
                "y": 2.266003131866455,
                "z": 0.8087740540504456
            },
            {
                "_id": "5c8faaaae37533453c77fb0f",
                "timestamp": "12000000",
                "x": 9.456435203552246,
                "y": 2.210968017578125,
                "z": 0.7848458290100098
            },
            {
                "_id": "5c8faaaae37533453c77fb10",
                "timestamp": "8000000",
                "x": 9.456435203552246,
                "y": 2.155933141708374,
                "z": 0.8350951075553894
            },
            {
                "_id": "5c8faaaae37533453c77fb11",
                "timestamp": "10000000",
                "x": 9.473184585571289,
                "y": 2.1774685382843018,
                "z": 0.8662018179893494
            },
            {
                "_id": "5c8faaaae37533453c77fb12",
                "timestamp": "14000000",
                "x": 9.504291534423828,
                "y": 2.163111686706543,
                "z": 0.8997012972831726
            },
            {
                "_id": "5c8faaaae37533453c77fb13",
                "timestamp": "6000000",
                "x": 9.53061294555664,
                "y": 2.115255117416382,
                "z": 0.8877372145652771
            },
            {
                "_id": "5c8faaaae37533453c77fb14",
                "timestamp": "21000000",
                "x": 9.604789733886719,
                "y": 2.50289249420166,
                "z": -0.22013968229293823
            },
            {
                "_id": "5c8faaaae37533453c77fb15",
                "timestamp": "2000000",
                "x": 9.315258979797363,
                "y": 2.59621262550354,
                "z": -0.3756731450557709
            },
            {
                "_id": "5c8faaaae37533453c77fb16",
                "timestamp": "7000000",
                "x": 9.317651748657227,
                "y": 2.2755744457244873,
                "z": 0.4139583110809326
            },
            {
                "_id": "5c8faaaae37533453c77fb17",
                "timestamp": "8000000",
                "x": 9.353543281555176,
                "y": 2.1343977451324463,
                "z": 1.1198409795761108
            },
            {
                "_id": "5c8faaaae37533453c77fb18",
                "timestamp": "12000000",
                "x": 9.181260108947754,
                "y": 2.108076810836792,
                "z": 1.2346965074539185
            },
            {
                "_id": "5c8faaaae37533453c77fb19",
                "timestamp": "16000000",
                "x": 9.119047164916992,
                "y": 2.1296122074127197,
                "z": 1.1413763761520386
            },
            {
                "_id": "5c8faaaae37533453c77fb1a",
                "timestamp": "12000000",
                "x": 9.166903495788574,
                "y": 2.115255117416382,
                "z": 1.0336993932724
            },
            {
                "_id": "5c8faaaae37533453c77fb1b",
                "timestamp": "2000000",
                "x": 9.28893756866455,
                "y": 2.2133610248565674,
                "z": 1.0097711086273193
            },
            {
                "_id": "5c8faaaae37533453c77fb1c",
                "timestamp": "7000000",
                "x": 9.442078590393066,
                "y": 2.3617160320281982,
                "z": 0.914058268070221
            },
            {
                "_id": "5c8faaaae37533453c77fb1d",
                "timestamp": "10000000",
                "x": 9.504291534423828,
                "y": 2.3784656524658203,
                "z": 0.8398807644844055
            },
            {
                "_id": "5c8faaaae37533453c77fb1e",
                "timestamp": "10000000",
                "x": 9.485149383544922,
                "y": 2.311466693878174,
                "z": 0.7417750358581543
            },
            {
                "_id": "5c8faaaae37533453c77fb1f",
                "timestamp": "13000000",
                "x": 9.42772102355957,
                "y": 2.328216314315796,
                "z": 0.7776673436164856
            }
        ]
    }
]

In [7]:
len(ds)

## Split the dataset into a test and training dataset
Each punch is represented by multiple rows of accelerometer values with the (current) timestamp and the punch label, hand and annotator information. To split the dataframe it is relevant to split by focusing only on the punch_id to split only complete punches (time series). The function <font color='red'>train_test_split_punchdataframe</font> extracts all (unique) punch_ids and splitts them random into test and training lists. After that the rows of each punchs are imported to the corresponding list. With the optional <font color='red'>seed</font> option it is guranteed that each split can result in the same punch constellation to reproduce/follow this notebook step by step. 

In [8]:
punchIdx = ds['punch_id'].unique()

In [9]:
len(punchIdx)

In [10]:
# returns an array of lists containing a list of the punch indizes to train and test
# first list element: train dataset
# second list element: test dataset
# arg 1: dataframe as dataframe object
# arg 2: trainsize as percentage of training data. e.g. 0.7
# arg3: seed . optional, for same results of random values at multiple test
def train_test_split_punchdataframe(dataframe, trainsize,seed=5):
    random.seed(seed)
    data = dataframe.copy()
    punchIdx = data['punch_id'].unique()
    list_in_cpy = punchIdx[:]
    random.shuffle(list_in_cpy)
    n = len(list_in_cpy)
    idx_train = list_in_cpy[:round(n*trainsize)]
    idx_test =  list_in_cpy[round(n*trainsize):]
    train_ds = data[data['punch_id'].isin(idx_train)]
    test_ds = data[data['punch_id'].isin(idx_test)]
    return [train_ds.reset_index(drop=True),test_ds.reset_index(drop=True)];

In [11]:
mixedPunches = train_test_split_punchdataframe(ds,.7)

In [12]:
train_dataset = mixedPunches[0]
test_dataset = mixedPunches[1]

In [13]:
train_dataset.head()

In [14]:
test_dataset.head()

## Checking whether the splitted datasets are balanced
For preventing the classifiers to learn one class "better" than other classes it is important to see if the train and test datasets contain nearly the same number of punches for each class.

In [15]:
train_dataset['label'].value_counts()

In [16]:
test_dataset['label'].value_counts()

<hr>
<center> <h1><font color="blue">Feature extraction: Create new test and train datasets with no time series character</font></h1> </center>
<hr>

In [17]:
# try to use the minimal feature settings
settings_minimal = MinimalFCParameters() # only a few basic features
settings_minimal

## Extract the chosen features from the test- and training dataset
For that the columns: label, hand and annotator are dropped because the features will only be extracted for the variables x,y,z of the accelerometer sensor. If a "bigger" feature-setting is used the extraction process can take a while. With the minimal feature setting used the extraction is done in at least one minute.

In [18]:
# extract features for test dataset
testds_for_extraction = test_dataset.drop(columns=['label','hand','annotator'])
test_dataset_extracted = extract_features(testds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

In [19]:
# extract features for train dataset
trainds_for_extraction = train_dataset.drop(columns=['label','hand','annotator'])
train_dataset_extracted = extract_features(trainds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

## Print the extracted feature values: train and test dataset
For now on the datasets containing the extracted features are used to learn the classifiers. At this point the time series classification problem is converted to a standard classification problem. So in the next steps the standard ML algorithms can be used for classification. Each time series is identified by the id column. So it is possible to connect the extracted features with the original time series (punch).

In [20]:
train_dataset_extracted.head()

In [21]:
test_dataset_extracted.head()

## Create y_test and y_train Series objects containing the information to predict
Depending on the classification label seperate y_test and y_train Series objects are created. In the following there are objects for all possible classifications of the smartPunch problem. Meaning there are y_test and y_train for predicting: <font color='blue'>hand, label and annotator<font/>

In [22]:
test_dataset_unique_label_id = test_dataset.drop_duplicates(subset='punch_id', keep='first', inplace=False)
test_dataset_unique_label_id.head()

In [23]:
y_test = pd.Series(data=test_dataset_unique_label_id['label'])
y_test.head()

In [24]:
train_dataset_unique_label_id = train_dataset.drop_duplicates(subset='punch_id', keep='first', inplace=False)
train_dataset_unique_label_id.head()

In [25]:
y_train = pd.Series(data=train_dataset_unique_label_id['label'])
y_train.head()

## create the X_train and X_test dataset containing the features

In [26]:
X_train = train_dataset_extracted
X_test = test_dataset_extracted

<hr>
<center> <h1><font color="blue">Visualization of the training dataset</font></h1>
    <h2>For verifying that the training dataset is balanced</h2>
</center>
<hr>

In [27]:
count_of_each_punch = np.array(y_train.value_counts())
punches = sorted(y_train.unique())

# Plot a pie chart for different punches
plt.rcParams.update({'figure.figsize': [20, 20], 'font.size': 24})
plt.pie(count_of_each_punch, labels = punches, autopct = '%0.2f')

<hr>
<center> <h1><font color="blue">Classification of the punches</font></h1> </center>
<h2>Use Linear SVC (with one-vs-the-rest), Standard SVC, Logistic Regression, KNN and Random Forest Classifier</h2>
<hr>

In [28]:
accuracy_scores = np.zeros(5)
clf_labels = ['Linear SVC (ovr)','Standard SVC', 'Logsitic Regression', 'KNN', 'Random Forest']

## Linear Support Vector Classifier

In [29]:
linSupp_Vectr_clf_punchtype = svm.LinearSVC()
linSupp_Vectr_clf_punchtype.fit(X_train, y_train) 
prediction = linSupp_Vectr_clf_punchtype.predict(X_test)
accuracy_scores[0] = accuracy_score(y_test, prediction)*100
print('Linear Vector Classifier accuracy (one-vs-rest): {}%'.format(accuracy_scores[0]))

## Standard support vector classifier

In [30]:
# Support Vector Classifier
stdSupp_Vectr_clf_punchtype = SVC().fit(X_train, y_train)
prediction = stdSupp_Vectr_clf_punchtype.predict(X_test)
accuracy_scores[1] = accuracy_score(y_test, prediction)*100
print('Support Vector Classifier accuracy: {}%'.format(accuracy_scores[1]))

## Logistic regression

In [31]:
# Logistic Regression
logistic_reggr_clf_punchtype = LogisticRegression().fit(X_train, y_train)
prediction = logistic_reggr_clf_punchtype.predict(X_test)
accuracy_scores[2] = accuracy_score(y_test, prediction)*100
print('Logistic Regression accuracy: {}%'.format(accuracy_scores[2]))

## K Nearest Neighbors

In [32]:
# K Nearest Neighbors
knn_clf_punchtype = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)
prediction = knn_clf_punchtype.predict(X_test)
accuracy_scores[3] = accuracy_score(y_test, prediction)*100
print('K Nearest Neighbors Classifier accuracy: {}%'.format(accuracy_scores[3]))

## Random Forest

In [33]:
# Random Forest
rndm_forest_clf_punchtype = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)
prediction = rndm_forest_clf_punchtype.predict(X_test)
accuracy_scores[4] = accuracy_score(y_test, prediction)*100
print('Random Forest Classifier accuracy: {}%'.format(accuracy_scores[4]))

## Overview of classifier prediction accuracy

In [34]:
colors = cm.rainbow(np.linspace(0, 1, 4))
plt.figure(figsize=(20,10))
plt.bar(clf_labels,
        accuracy_scores,
        color = colors)
plt.xlabel('Classifiers')
plt.ylabel('Accuracy')
plt.title('Accuracy overview for label prediction')

# Feature extraction for learning: hand

In [35]:
train_dataset = mixedPunches[0]
test_dataset = mixedPunches[1]

In [36]:
# try to use the minimal feature settings
settings_minimal = MinimalFCParameters() # only a few basic features
settings_minimal

## extract the features for learning and training dataset

In [37]:
# extract features for test dataset
testds_for_extraction = test_dataset.drop(columns=['label','hand','annotator'])
test_dataset_extracted = extract_features(testds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

In [38]:
# extract features for train dataset
trainds_for_extraction = train_dataset.drop(columns=['label','hand','annotator'])
train_dataset_extracted = extract_features(trainds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

## create y_test and y_train dataset containing the hand information

In [39]:
test_dataset_unique_label_id = test_dataset.drop_duplicates(subset='punch_id', keep='first', inplace=False)
test_dataset_unique_label_id.head()

In [40]:
y_test = pd.Series(data=test_dataset_unique_label_id['hand'])

In [41]:
train_dataset_unique_label_id = train_dataset.drop_duplicates(subset='punch_id', keep='first', inplace=False)
train_dataset_unique_label_id.head()

In [42]:
y_train = pd.Series(data=train_dataset_unique_label_id['hand'])
y_train.head()

## create the X_train and X_test dataset containing the features

In [43]:
X_train = train_dataset_extracted
X_test = test_dataset_extracted

## classify the hand of the subject

In [44]:
accuracy_scores_for_hand = np.zeros(5)
clf_for_hand = ['Linear SVC (ovr)','Standard SVC', 'Logsitic Regression', 'KNN', 'Random Forest']

## Linear Support Vector Classifier

In [45]:
linSupp_Vectr_clf_hand = svm.LinearSVC()
linSupp_Vectr_clf_hand.fit(X_train, y_train) 
prediction = linSupp_Vectr_clf_hand.predict(X_test)
accuracy_scores_for_hand[0] = accuracy_score(y_test, prediction)*100
print('Linear Vector Classifier accuracy (one-vs-rest): {}%'.format(accuracy_scores_for_hand[0]))

## Standard support vector classifier

In [46]:
# Support Vector Classifier
stdSupp_Vectr_clf_hand = SVC().fit(X_train, y_train)
prediction = stdSupp_Vectr_clf_hand.predict(X_test)
accuracy_scores_for_hand[1] = accuracy_score(y_test, prediction)*100
print('Support Vector Classifier accuracy: {}%'.format(accuracy_scores_for_hand[1]))

## Logistic regression

In [47]:
# Logistic Regression
logistic_reggr_clf_hand = LogisticRegression().fit(X_train, y_train)
prediction = logistic_reggr_clf_hand.predict(X_test)
accuracy_scores_for_hand[2] = accuracy_score(y_test, prediction)*100
print('Logistic Regression accuracy: {}%'.format(accuracy_scores_for_hand[2]))

## K Nearest Neighbors

In [48]:
# K Nearest Neighbors
knn_clf_hand = KNeighborsClassifier(n_neighbors=5)
knn_clf_hand.fit(X_train, y_train)
prediction = knn_clf_hand.predict(X_test)
accuracy_scores_for_hand[3] = accuracy_score(y_test, prediction)*100
print('K Nearest Neighbors Classifier accuracy: {}%'.format(accuracy_scores_for_hand[3]))

## Random Forest

In [49]:
# Random Forest
rndm_forest_clf_hand = RandomForestClassifier(n_estimators = 100)
rndm_forest_clf_hand.fit(X_train, y_train)
prediction = rndm_forest_clf_hand.predict(X_test)
accuracy_scores_for_hand[4] = accuracy_score(y_test, prediction)*100
print('Random Forest Classifier accuracy: {}%'.format(accuracy_scores_for_hand[4]))

## Prediction accuracy

In [50]:
colors = cm.rainbow(np.linspace(0, 1, 4))
plt.figure(figsize=(20,10))
plt.bar(clf_for_hand,
        accuracy_scores_for_hand,
        color = colors)
plt.xlabel('Classifiers')
plt.ylabel('Accuracy')
plt.title('Accuracy overview for hand-prediction')

# Feature extraction for learning: Annotator

In [51]:
train_dataset = mixedPunches[0]
test_dataset = mixedPunches[1]

In [52]:
# try to use the minimal feature settings
settings_minimal = MinimalFCParameters() # only a few basic features
settings_minimal

## extract the features for learning and training dataset

In [53]:
# extract features for test dataset
testds_for_extraction = test_dataset.drop(columns=['label','hand','annotator'])
test_dataset_extracted = extract_features(testds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

In [54]:
# extract features for train dataset
trainds_for_extraction = train_dataset.drop(columns=['label','hand','annotator'])
train_dataset_extracted = extract_features(trainds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

## create y_test and y_train dataset containing the hand information

In [55]:
test_dataset_unique_label_id = test_dataset.drop_duplicates(subset='punch_id', keep='first', inplace=False)
test_dataset_unique_label_id.head()

In [56]:
y_test = pd.Series(data=test_dataset_unique_label_id['annotator'])

In [57]:
train_dataset_unique_label_id = train_dataset.drop_duplicates(subset='punch_id', keep='first', inplace=False)
train_dataset_unique_label_id.head()

In [58]:
y_train = pd.Series(data=train_dataset_unique_label_id['annotator'])
y_train.head()

## create the X_train and X_test dataset containing the features

In [59]:
X_train = train_dataset_extracted
X_test = test_dataset_extracted

In [60]:
X_train.head()

## classify the subject

In [61]:
accuracy_scores_for_subject = np.zeros(5)
clf_for_subject = ['Linear SVC (ovr)','Standard SVC', 'Logsitic Regression', 'KNN', 'Random Forest']

## Linear Support Vector Classifier

In [62]:
linSupp_Vectr_clf_annotator = svm.LinearSVC().fit(X_train, y_train) 
prediction = linSupp_Vectr_clf_annotator.predict(X_test)
accuracy_scores_for_subject[0] = accuracy_score(y_test, prediction)*100
print('Linear Vector Classifier accuracy (one-vs-rest): {}%'.format(accuracy_scores_for_subject[0]))

## Standard support vector classifier

In [63]:
# Support Vector Classifier
stdSupp_Vectr_clf_annotator = SVC().fit(X_train, y_train)
prediction = stdSupp_Vectr_clf_annotator.predict(X_test)
accuracy_scores_for_subject[1] = accuracy_score(y_test, prediction)*100
print('Support Vector Classifier accuracy: {}%'.format(accuracy_scores_for_subject[1]))

## Logistic regression

In [64]:
# Logistic Regression
logistic_reggr_clf_annotator = LogisticRegression().fit(X_train, y_train)
prediction = logistic_reggr_clf_annotator.predict(X_test)
accuracy_scores_for_subject[2] = accuracy_score(y_test, prediction)*100
print('Logistic Regression accuracy: {}%'.format(accuracy_scores_for_subject[2]))

## K Nearest Neighbors

In [65]:
# K Nearest Neighbors
knn_clf_annotator = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)
prediction = knn_clf_annotator.predict(X_test)
accuracy_scores_for_subject[3] = accuracy_score(y_test, prediction)*100
print('K Nearest Neighbors Classifier accuracy: {}%'.format(accuracy_scores_for_subject[3]))

## Random Forest

In [66]:
# Random Forest
rndm_forest_clf_annotator = RandomForestClassifier(n_estimators = 50).fit(X_train, y_train)
prediction = rndm_forest_clf_annotator.predict(X_test)
accuracy_scores_for_subject[4] = accuracy_score(y_test, prediction)*100
print('Random Forest Classifier accuracy: {}%'.format(accuracy_scores_for_subject[4]))

## Prediction accuracy

In [67]:
colors = cm.rainbow(np.linspace(0, 1, 4))
plt.figure(figsize=(20,10))
plt.bar(clf_for_subject,
        accuracy_scores_for_subject,
        color = colors)
plt.xlabel('Classifiers')
plt.ylabel('Accuracy')
plt.title('Accuracy overview for annotator prediction')

<hr>
<center> <h1><font color="blue">Save the Models for reusing later</font></h1> </center>
<h2>Save the trained classification models for reuse in other scripts.</h2>
<hr>

In [68]:
import pickle

## Store the models for the punch classification

### Linear SVC

In [122]:
pkl_filename = "model_linSupp_Vectr_clf_punchtype_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(linSupp_Vectr_clf_punchtype, file)

### Standard SVC

In [123]:
pkl_filename = "model_stdSupp_Vectr_clf_punchtype_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(stdSupp_Vectr_clf_punchtype, file)

### Logistic Regression

In [124]:
pkl_filename = "model_logistic_reggr_clf_punchtype_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(logistic_reggr_clf_punchtype, file)

### KNN

In [125]:
pkl_filename = "model_knn_clf_punchtype_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(knn_clf_punchtype, file)

### Random Forest

In [126]:
pkl_filename = "model_rndm_forest_clf_punchtype_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(rndm_forest_clf_punchtype, file)

## Store the models for the hand classification

### Linear SVC

In [127]:
pkl_filename = "model_linSupp_Vectr_clf_hand_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(linSupp_Vectr_clf_hand, file)

### Standard SVC

In [128]:
pkl_filename = "model_stdSupp_Vectr_clf_hand_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(stdSupp_Vectr_clf_hand, file)

### Logistic Regression

In [129]:
pkl_filename = "model_logistic_reggr_clf_hand_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(logistic_reggr_clf_hand, file)

### KNN

In [130]:
pkl_filename = "model_knn_clf_hand_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(knn_clf_hand, file)

### Random Forest

In [131]:
pkl_filename = "model_rndm_forest_clf_hand_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(rndm_forest_clf_hand, file)

## Store the models for the annotator classification

### Linear SVC

In [132]:
pkl_filename = "model_linSupp_Vectr_clf_annotator_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(linSupp_Vectr_clf_annotator, file)

### Standard SVC

In [133]:
pkl_filename = "model_stdSupp_Vectr_clf_annotator_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(stdSupp_Vectr_clf_annotator, file)

### Logistic Regression

In [134]:
pkl_filename = "model_logistic_reggr_clf_annotator_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(logistic_reggr_clf_annotator, file)

### KNN

In [135]:
pkl_filename = "model_knn_clf_annotator_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(knn_clf_annotator, file)

### Random Forest

In [136]:
pkl_filename = "model_rndm_forest_clf_annotator_periodMS2000_sampleUS1000_TrainSize70.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(rndm_forest_clf_annotator, file)